## Imports

In [1]:
import H5toDataframe as readH5
import influxdb_client as influx
from influxdb_client.client.write_api import SYNCHRONOUS
import conversionServices as ConvServ
import datetime as dt

## API configuration

In [6]:
bucket = "43ecbfaf3070c072" #test6
org = "H493f0c3e5bf84239"
token = "giJmgHGcBaEt8LvH1kZbfpwrNU3_eYDg32MXqbfkZNyuJHxfyOHTFAyYBsOs7M2s-WKRneuGu849FTT3CNdCHg=="
# Store the URL of your InfluxDB instance
url="https://westeurope-1.azure.cloud2.influxdata.com/"

client = influx.InfluxDBClient(
    url=url,
    token=token,
    org=org
)

write_api = client.write_api(write_options=influx.client.write_api.SYNCHRONOUS)

## Read data from HDF5

In [2]:
df = readH5.convertAllH5Files()

unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'


In [ ]:
# Query script
query_api = client.query_api()
query = ('from(bucket:"test5")\
|> range(start: 0)\
|> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")')

result = query_api.query(org=org, query=query)
# results = []
# for table in result:
#     for record in table.records:
#         results.append((record.get_field(), record.get_value()))
        # print(record)

print(result)


In [7]:
for name, (frame, attrs) in df.items():
    year = ConvServ.getYearFromDf(frame) # extract year from timestamp
    avg_velo = ConvServ.dataframe_column_average(frame, "velocity") # calculate average velocity
    points = []

    for row in frame.iterrows():

        # print(row[1]["timestamp"])
        # ts = dt.datetime.fromtimestamp(row[1]["timestamp"])

    try:

        p = (influx.Point(name)
        .time(row[1]["timestamp"]) # Muss bereits als datetime vorliegen
        .tag("instrument", attrs["instrument"])
        .tag("cofiguration", attrs["configuration"])
        .tag("year", year)
        .tag("avg_velocity", avg_velo) 
        .field("defect_channel", row[1]["defect_channel"])
        .field("distance", row[1]["distance"])
        .field("magnetization", row[1]["magnetization"])
        .field("velocity", row[1]["velocity"])
        .field("wall_thickness", row[1]["wall_thickness"]))
        
        points.append(p) 
    
    # write_api.write(bucket=bucket, org=org, record=points)

TypeError: Could not convert 0.63410396465900990.67468771968407560.65659208844233470.64156823582485310.65924227402302480.64258681738013020.60181421435966210.6890062842346250.64387553939481610.6907975126399230.67267368334880650.69992895627821110.66969258548629470.66094807463030360.69756981342359680.70140834468383680.63630399197358420.58901348120709950.63468865621099910.6179205435713510.68891285840004610.61995312760776260.69026078444719810.58294670906965360.66551377584528940.60922585898055660.68993120085383030.69856412942047830.61379537784927010.65711050598362750.65199895851750660.561220001220647648.116075719940210.64874498400617740.63638420271294680.66296795780143280.69109904989410510.6008040583042470.66773207871822470.64502072214541140.72163905468677560.67145648095672430.65205892721209780.68725105660650140.70140992263241110.67173091320196950.65952815355881620.69199130019381770.76016298210690620.75281700993043280.70238641007598030.77572118341325060.75442914301905710.85866826155283540.94